<a href="https://colab.research.google.com/github/Khoroshai/ForgeColabServer/blob/main/ForgeColabServer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Forge Minecraft Server
*This method is inspired from MineColab (https://github.com/thecoder-001/MineColab)*

*It uses free ressources thanks to Google, do not abuse of it.*

This method uses zipped saves files and zipped server folder to reduce transfer time and numbers with gdrive which improves greatly the process and might avoid the famous "Transport endpoint not connected" error caused by a kind of desync with gdrive, afaik.

The world files and logs are saved to gdrive every 5 minutes.

*Tested on 1.18.2 with around 80 mods*

# Requirements
- An open tab that will stay focused
- A base vanilla minecraft server
- A base Forge server
- Mods
- A Playit.gg account (free)

# How to use
1) Put the base vanilla and Forge server in a folder and add the mods to it.

2) *Zip* this folder and upload it somewhere on your gdrive.

3) *Change* the Names, Paths and the Forge command in **Initialization**.

4) *Run* only one time the first cell to set everything up.

5) (using Playit.gg) Follow the link to activate the tunnel. (You might wait a few seconds, just follow the instruction) In case it is not the first tunnel you open, you can click on *Create Tunnel* and then stop the creation as only the agent is necessary. **The URL to the server is on Playit.gg**. If the URL doesn't work, the IP should work.

6) *Launch* the **Start Server** cell as you wish. Stopping it will gently close the server. (Could raise an error but trying a second time will close normally with saving worlds, etc... in normal cases)

7) *Run* the **Stop Auto-saves** cell when the server is stopped. You can run it while the server is up to make it auto-run when the server stops.

8) (optional) You can run the very last cell to save any changes made to the server. (properties, config...)

# Additional informations

*MineColab* has a cell to create the base server on gdrive. You can run it and then download the folder to continue the creation (Forge + mods + zipping)

To create the base Forge server, as of August 2023, you can execute a jar file found on their official website.

The *run.sh* file, that you'll get with the Forge base, contains a command which includes a version that you'll compare with *forgeCommand* at **Initialization**.

The tunnel used in this Notebook is Playit but you can easily copy-paste Ngrok or Argo from *MineColab*. Just pay attention to the command slighty modified here.

# Known issues

- The **Stop Auto-saves** cell takes up to 5 minutes (saving interval) to complete. Run, stop, run is a work around by raising an error.
Possible solution : use a subprocess to interrupt it more easily.

- Sometimes the server console stops allowing inputs. This happens when the bottom timer also stops counting the running time despite the server being up.

# To do

- Add Ngrok and Argo.

# Initialization

In [ ]:
from google.colab import drive
import shutil
import zipfile
import os
import re
import json

# ___Names&Paths setup___
pathToDrive = '/content/drive/My Drive/Minecraft-server/' #@param {type:"string"}
worldName = 'test' #@param {type:"string"}

# Please, make an archive of the whole server on the pathToDrive for easy transfer
serverArchiveName = 'MC.zip' #@param {type:"string"}

# Should be the same name as the drive one
colabserver_folder = '/content/Minecraft-server/' #@param {type:"string"}
Overworld = worldName
Nether = worldName + '_nether'
End = worldName + '_the_end'

# Check the command in run.sh
forgeCommand = "@libraries/net/minecraftforge/forge/1.18.2-40.2.10/unix_args.txt" #@param {type:"string"}


# ___Import everything from gdrive___
#drive.mount('/content/drive')
driveserver_archive = pathToDrive + serverArchiveName
with zipfile.ZipFile(driveserver_archive, 'r') as zip_ref :
  zip_ref.extractall(colabserver_folder)
print('Importing server done.')
try :
  driveworld = pathToDrive + worldName + '.zip'
  with zipfile.ZipFile(driveworld, 'r') as zip_ref :
    zip_ref.extractall(colabserver_folder + worldName)
  print('Importing Overworld done.')

  drivenether = pathToDrive + Nether + '.zip'
  with zipfile.ZipFile(drivenether, 'r') as zip_ref :
    zip_ref.extractall(colabserver_folder + Nether)
  print('Importing Nether done.')

  driveend = pathToDrive + End + '.zip'
  with zipfile.ZipFile(driveend, 'r') as zip_ref :
    zip_ref.extractall(colabserver_folder + End)
  print('Importing End done.')
except FileNotFoundError as NoWorldYet:
  print('No world to import yet.')


updated_lines = []
with open(colabserver_folder + 'server.properties', 'r') as file:
    for line in file:
        if 'level-name' in line:
            updated_lines.append(f'level-name={worldName}\n')
        else:
            updated_lines.append(line)

with open(colabserver_folder + 'server.properties', 'w') as file:
    file.writelines(updated_lines)




# ___Prepare the environment___
# Update the package lists
!sudo apt update &>/dev/null && echo "apt cache successfully updated" || echo "apt cache update failed, you might receive stale packages"
# Install OpenJDK 17
# !wget -qO - https://adoptopenjdk.jfrog.io/adoptopenjdk/api/gpg/key/public | sudo apt-key add -
# !sudo add-apt-repository --yes https://adoptopenjdk.jfrog.io/adoptopenjdk/deb/ &>/dev/null || echo "Failed to add repo. Still can be ignored if openjdk17 gets installed."
!sudo apt-get install openjdk-17-jre-headless &>/dev/null && echo "Yay! Openjdk17 has been successfully installed." || echo "Failed to install OpenJdk17."
# Perform java version check
java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'
if java_ver[0] == "17" :
  print("Openjdk17 is working correctly, you are good to go.")
else:
  print("Openjdk17 doesn't seems to be installed or isn't working, falling back to java", java_ver[0], ". You might experience reduced performance. Minecraft 1.17 and above might fail to launch.")

# Change directory to the Minecraft server folder
%cd {colabserver_folder}

# Import config file.
if os.path.isfile("colabconfig.json"):
  colabconfig = json.load(open("colabconfig.json"))
else:
  colabconfig = {"server_type": "generic"} # using default, if config doesn't exists.
  json.dump(colabconfig, open("colabconfig.json",'w'))

# Server jar names.
jar_list = {'paper': 'server.jar', 'fabric': 'fabric-server-launch.jar', 'generic': 'server.jar', 'bukkit': 'craftbukkit.jar'}
jar_name = jar_list[colabconfig["server_type"]]

# Java arguments.
if colabconfig["server_type"] == "paper":
  server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"
else:
  server_flags = "" # aiker's flags might negatively impact performance on non-paper servers.
memory_allocation = "-Xmx8000M -Xms100M"

# Chose the tunnle service you want to use
# Available options: ngrok, argo, playit
tunnel_service = "ngrok"
print("Proceeding to use", tunnel_service)

if tunnel_service == "playit":
  ! curl -SsL https://playit-cloud.github.io/ppa/key.gpg | sudo apt-key add -
  ! sudo curl -SsL -o /etc/apt/sources.list.d/playit-cloud.list https://playit-cloud.github.io/ppa/playit-cloud.list
  ! sudo apt update &>/dev/null && sudo apt install playit &>/dev/null && echo "Playit.gg installed" || echo "Failed to install playit"
  print('Starting tunnel')
  ! playit > playit.log 2>&1 &
# Output the line with the URL to open the tunnel
  with open('playit.log', 'r') as tunnelfile:
    for _ in range(3):
        tunnelfile.readline()
    third_ligne = tunnelfile.readline()
    print(third_ligne)

elif tunnel_service == "ngrok":
  !pip -q install pyngrok
  from pyngrok import conf, ngrok

  # Ask for the ngrok authtoken
  print("Get your authtoken from https://dashboard.ngrok.com/auth")
  import getpass
  authtoken = getpass.getpass()  # input your Ngrok auth token everytime you run the cell or simply replace "getpass.getpass()" with your token in "double quotes"
  ! ngrok authtoken $authtoken # login to ngrok

  # Sets default ngrok region
  conf.get_default().region = 'eu'  # Change this to whichever region you want

  # Connect to ngrok
  url = ngrok.connect(25565, 'tcp')
  print('Your server address is ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', ''))
  !ngrok tcp --domain=apparently-brief-polliwog.ngrok-free.app 80
  print('Starting server...')
  !java $memory_allocation $server_flags -jar $jar_name nogui #@libraries/net/minecraftforge/forge/1.18.2-40.2.10/unix_args.txt nogui

elif tunnel_service == "argo":
  # Download & make argo executable
  !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
  !chmod +x cloudflared-linux-amd64
  print('Starting tunnel')
  !./cloudflared-linux-amd64 tunnel --url tcp://127.0.0.1:25565 & java $memory_allocation $server_flags -jar $jar_name nogui



# Start Server

In [ ]:
import threading
import time
import shutil
from datetime import datetime

# ___Launch the server with an auto-save to gdrive___
Running = True

# Auto-saving function
def run_saveToDrive_task():
    while Running == True:
        Timestamp = datetime.now().strftime("%H:%M:%S")
        print(f"[{Timestamp}] [Auto-save] Saving World & logs to {pathToDrive}")
        shutil.make_archive(pathToDrive + Overworld, 'zip', Overworld)
        shutil.make_archive(pathToDrive + Nether, 'zip', Nether)
        shutil.make_archive(pathToDrive + End, 'zip', End)
        shutil.copytree('logs', pathToDrive + 'logs', dirs_exist_ok=True)
        time.sleep(300)  # Wait 5 minutes

# Creating thread
Save_thread = threading.Thread(target=run_saveToDrive_task)
# Starting thread
Save_thread.start()
# Starting server
! java $memory_allocation $server_flags $forgeCommand nogui
# Start Vanilla server
#! java $memory_allocation $server_flags -jar $jar_name nogui

**Stop Auto-saves or Do manual save**

In [ ]:
# Stop the auto-saving (may take some time) | Rerun this cell for manual saving the worlds & logs
Running = False
Save_thread.join()

# Saves a last time after shutting down
shutil.make_archive(pathToDrive + Overworld, 'zip', Overworld)
shutil.make_archive(pathToDrive + Nether, 'zip', Nether)
shutil.make_archive(pathToDrive + End, 'zip', End)
shutil.copytree('logs', pathToDrive + 'logs', dirs_exist_ok=True)
print("Done.")

**Save the whole server if you made changes**

In [ ]:
import shutil
# Saving the whole server to drive
shutil.make_archive(pathToDrive + 'MC', 'zip', colabserver_folder)

In [ ]:
! java $memory_allocation $server_flags -jar $jar_name nogui